In [1]:
from collections import OrderedDict
from conllu import parse
from enum import Enum
import stanza
import pymorphy2
import tokenize_uk
import os

In [2]:
stanza_nlp = stanza.Pipeline(lang='uk')

2020-05-12 17:37:13 INFO: Loading these models for language: uk (Ukrainian):
| Processor | Package |
-----------------------
| tokenize  | iu      |
| mwt       | iu      |
| pos       | iu      |
| lemma     | iu      |
| depparse  | iu      |

2020-05-12 17:37:13 INFO: Use device: cpu
2020-05-12 17:37:13 INFO: Loading: tokenize
2020-05-12 17:37:13 INFO: Loading: mwt
2020-05-12 17:37:13 INFO: Loading: pos
2020-05-12 17:37:14 INFO: Loading: lemma
2020-05-12 17:37:14 INFO: Loading: depparse
2020-05-12 17:37:16 INFO: Done loading processors!


In [3]:
script_path = os.path.abspath('__file__') 
path_list = script_path.split(os.sep)
script_directory = path_list[0:len(path_list)-5]
rel_path = "UD_Ukrainian-IU"
PATH = "/".join(script_directory[:4]) + "/" + rel_path
 
with open(PATH + "/uk_iu-ud-train.conllu", "r") as f:
    train_trees = parse(f.read())

In [4]:
corpus = ["Цінова лихоманка у царині енергоресурсів засвідчує, що світова економіка переживає не найкращі часи.", 
          "Проте усвідомлення цього факту не завадило верховному представнику ЄС у закордонних справах та політиці безпеки Жозепу Боррелю під час спілкування з Дмитром Кулебою наголосити на зацікавленості Європейського Союзу у співпраці з Україною та готовності здійснити відкладений візит до України тільки-но для цього з’явиться можливість.",
          "Нагадаю, що пан Боррель мав намір відвідати Донбас, що протягом останніх років стало ледь не обов’язковою частиною для перебування в Україні поважних іноземних гостей.",
          "Міністри закордонних справ країн ЄС також продемонстрували готовність до розвитку співпраці з Україною, а Європейська Комісія оголосила про готовність виділити для України 1,2 мільярди для боротьби з наслідками коронавірусу.",  
          "Російська Федерація також демонструє активність щодо України, проте дещо іншого порядку.", 
          "У Москві свідомо ігнорують очевидне: словосполучення «російський гуманітарний конвой» з 2014 року стало синонімом незаконного втручання та підтримки бойовиків."]

In [5]:
# will be taken for a golden parse
def golden_tree(sentence):
    transformed_tree = []
    doc = stanza_nlp(sentence)
    for sent in doc.sentences:
        for word in sent.words:
            transformed_tree.append(OrderedDict([('id', int(word.id)), ('form',  word.text), ('lemma',word.lemma),
                                     ('upostag', word.upos), ('xpostag', None),
                                     ('feats', word.feats), ('head', int(word.head)), ('deprel', None),
                                     ('deps', None), ('misc', None)]))
    return transformed_tree

## Version I. 
- golden - залежності, які проставив парсер stanz-и
- Порівнюю роботу свого парсера на реченнях, які привела до необхідного вигляду з допомогою pymorphy2
- Граматичні ознаки зліплюю (бо набір з conllu - 'feats', OrderedDict([('Animacy', 'Inan'), ('Case', 'Nom'), ('Gender', 'Neut'), ('Number', 'Sing')]), а в pymorhpy - 'feats', OrderedDict([('NOUN,', 'inan femn,nomn')]), просто кодувати по одній не допоможе, їх потрібно привести до одного вигляду.

In [6]:
morph = pymorphy2.MorphAnalyzer(lang='uk')

DET = ['будь-який', 'ваш', 'ввесь', 'весь', 'все', 'всенький', 'всякий',
       'всілякий', 'деякий', 'другий', 'жадний', 'жодний', 'ин.', 'ін.',
       'інакший', 'інш.', 'інший', 'їх', 'їхній', 'її', 'його', 'кожний',
       'кожній', 'котрий', 'котрийсь', 'кілька', 'мій', 'наш', 'небагато',
       'ніякий', 'отакий', 'отой', 'оцей', 'сам', 'самий', 'свій', 'сей',
       'скільки', 'такий', 'тамтой', 'твій', 'те', 'той', 'увесь', 'усякий',
       'усілякий', 'це', 'цей', 'чий', 'чийсь', 'який', 'якийсь']

PREP = ["до", "на"]

mapping = {"ADJF": "ADJ", "ADJS": "ADJ", "COMP": "ADJ", "PRTF": "ADJ",
           "PRTS": "ADJ", "GRND": "VERB", "NUMR": "NUM", "ADVB": "ADV",
           "NPRO": "PRON", "PRED": "ADV", "PREP": "ADP", "PRCL": "PART"}

def normalize_pos(word):
    if word.tag.POS == "CONJ":
        if "coord" in word.tag:
            return "CCONJ"
        else:
            return "SCONJ"
    elif "PNCT" in word.tag:
        return "PUNCT"
    elif word.normal_form in PREP:
        return "PREP"
    elif word.normal_form in DET:
        return "DET"
    else:
        return mapping.get(word.tag.POS, word.tag.POS)

In [7]:
# transform test sentences
def convert_sentence_with_pymorphy_feats(sentence):
    tokenized_sent = tokenize_uk.tokenize_words(sentence)
    morphs = [morph.parse(word) for word in tokenized_sent]
    converted_tree = []
    
    for idx, mrph in enumerate(morphs):
        w = mrph[0]
        tagset = str(w.tag)
        pos = tagset[:5]
        # an empty feature set: tag=OpencorporaTag('PRCL')
        if len(tagset) <= 4:
            feats = str(False)
        else:
            feats = tagset[5:]
        features = OrderedDict([(pos, feats)])
        converted_tree.append(OrderedDict([('id', idx + 1), ('form', w.word), ('lemma', w.normal_form),
                                         ('upostag', normalize_pos(w)), ('xpostag', None),
                                         ('feats', features), ('head', None), ('deprel', None),
                                         ('deps', None), ('misc', None)]))
        
    return converted_tree

In [8]:
class Actions(str, Enum):
    SHIFT = "shift"
    REDUCE = "reduce"
    RIGHT = "right"
    LEFT = "left"
    
def oracle(stack, top_queue, relations):
    """
    Make a decision on the right action to do.
    """
    top_stack = stack[-1]
    # check if both stack and queue are non-empty
    if top_stack and not top_queue:
        return Actions.REDUCE
    # check if there are any clear dependencies
    elif top_queue["head"] == top_stack["id"]:
        return Actions.RIGHT
    elif top_stack["head"] == top_queue["id"]:
        return Actions.LEFT
    # check if we can reduce the top of the stack
    elif top_stack["id"] in [i[0] for i in relations] and \
         (top_queue["head"] < top_stack["id"] or \
          [s for s in stack if s["head"] == top_queue["id"]]):
        return Actions.REDUCE
    # default option
    else:
        return Actions.SHIFT

In [9]:
ROOT = OrderedDict([('id', 0), ('form', 'ROOT'), ('lemma', 'ROOT'), ('upostag', 'ROOT'),
                    ('xpostag', None), ('feats', None), ('head', None), ('deprel', None),
                    ('deps', None), ('misc', None)])

In [10]:
def extract_features(stack, queue):
    features = dict()

    if len(stack) > 0:
        stack_top = stack[-1]
        features["s0-word"] = stack_top["form"]
        features["s0-lemma"] = stack_top["lemma"]
        features["s0-tag"] = stack_top["upostag"]
         # фічі для слова в стеці
        if stack_top["feats"] != None:
            features["s0-feats"] = "_".join([f for f in stack_top["feats"].values()])
        else:
            features["s0-feats"] = False
    if len(stack) > 1:
        features["s1-tag"] = stack[-2]["upostag"]
        if stack[-2]["feats"] != None:
            features["s1-feats"] = "_".join([f for f in stack[-2]["feats"].values()])
        else:
            features["s1-feats"] = False
    if queue:
        queue_top = queue[0]
        features["q0-word"] = queue_top["form"]
        features["q0-lemma"] = queue_top["lemma"]
        features["q0-tag"] = queue_top["upostag"]
        # фічі для 1-го слова в черзі
        if queue_top["feats"] != None:
            features["q0-feats"] = "_".join([f for f in queue_top["feats"].values()])
        else:
            features["q0-feats"] = False
    if len(queue) > 1:
        queue_next = queue[1]
        features["q1-word"] = queue_next["form"]
        features["q1-tag"] = queue_next["upostag"]
        # фічі для 2-го слова в черзі
        if queue_next["feats"] != None:
            features["q1-feats"] = "_".join([f for f in queue_next["feats"].values()])
        else:
            features["q1-feats"] = False
    if len(queue) > 2:
        features["q2-tag"] = queue[2]["upostag"]
    if len(queue) > 3:
        features["q3-tag"] = queue[3]["upostag"]
    return features

In [11]:
def get_data(tree):
    features, labels = [], []
    stack, queue, relations = [ROOT], tree[:], []
    
    while queue or stack:
        action = oracle(stack if len(stack) > 0 else None,
                       queue[0] if len(queue) > 0 else None,
                       relations)
        features.append(extract_features(stack, queue))
        labels.append(action.value)
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            stack.append(queue.pop(0))
        else:
            print("Unknown action.")
    return features, labels

In [12]:
train_features, train_labels = [], []
for tree in train_trees:
    tree_features, tree_labels = get_data([t for t in tree if type(t["id"])==int])
    train_features += tree_features
    train_labels += tree_labels

print(len(train_features), len(train_labels))

190298 190298


In [13]:
from sklearn.feature_extraction import DictVectorizer 
from sklearn.linear_model import LogisticRegression

In [14]:
vectorizer = DictVectorizer()
vec = vectorizer.fit(train_features)
print("\nTotal number of features:", len(vec.get_feature_names()))


Total number of features: 115303


In [15]:
train_features_vectorized = vec.transform(train_features)
print(len(train_features_vectorized.toarray()))

190298


In [16]:
lrc = LogisticRegression(random_state=42, solver="saga", multi_class="multinomial", max_iter=1000, verbose=1)
lrc.fit(train_features_vectorized, train_labels)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 644 epochs took 188 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.1min finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='saga', tol=0.0001, verbose=1,
                   warm_start=False)

In [17]:
def dep_parse(sent, oracle, vectorizer):
    stack, queue, relations = [ROOT], sent[:], []
    while queue or stack:
        if stack and not queue:
            stack.pop()
        else:
            features = extract_features(stack, queue)
            action = oracle.predict(vectorizer.transform([features]))[0]
            if action == Actions.SHIFT:
                stack.append(queue.pop(0))
            elif action == Actions.REDUCE:
                stack.pop()
            elif action == Actions.LEFT:
                relations.append((stack[-1]["id"], queue[0]["id"]))
                stack.pop()
            elif action == Actions.RIGHT:
                relations.append((queue[0]["id"], stack[-1]["id"]))
                stack.append(queue.pop(0))
            else:
                print("Unknown action.")
    return sorted(relations)

In [19]:
total, tp, full_match = 0, 0, 0
for sentence in corpus:
    transf_tree = convert_sentence_with_pymorphy_feats(sentence)
    g_tree = golden_tree(sentence)
    tree = [t for t in transf_tree if type(t["id"])==int]
    golden = [(node["id"], node["head"]) for node in g_tree]
    predicted = dep_parse(tree, lrc, vec)
    total += len(tree)
    print("Sentence:", [i["form"]+"_"+str(i["id"]) for i in transf_tree])
    print("Golden:", golden)
    print("Predicted:", predicted)
    print("=================================================================")
    tp += len(set(golden).intersection(set(predicted)))
    if set(golden) == set(predicted):
        full_match += 1

print("Total:", total)
print("Correctly defined:", tp)
print("UAS:", round(tp/total, 2))
print("Full match:", round(full_match/len(corpus), 2))

Sentence: ['цінова_1', 'лихоманка_2', 'у_3', 'царині_4', 'енергоресурсів_5', 'засвідчує_6', ',_7', 'що_8', 'світова_9', 'економіка_10', 'переживає_11', 'не_12', 'найкращі_13', 'часи_14', '._15']
Golden: [(1, 2), (2, 6), (3, 4), (4, 2), (5, 4), (6, 0), (7, 11), (8, 11), (9, 10), (10, 11), (11, 6), (12, 13), (13, 14), (14, 11), (15, 6)]
Predicted: [(1, 2), (2, 0), (2, 6), (3, 4), (4, 2), (5, 4), (6, 0), (7, 11), (8, 11), (9, 10), (10, 11), (11, 6), (12, 14), (13, 14), (14, 11), (15, 6)]
Sentence: ['проте_1', 'усвідомлення_2', 'цього_3', 'факту_4', 'не_5', 'завадило_6', 'верховному_7', 'представнику_8', 'єс_9', 'у_10', 'закордонних_11', 'справах_12', 'та_13', 'політиці_14', 'безпеки_15', 'жозепу_16', 'боррелю_17', 'під_18', 'час_19', 'спілкування_20', 'з_21', 'дмитром_22', 'кулебою_23', 'наголосити_24', 'на_25', 'зацікавленості_26', 'європейського_27', 'союзу_28', 'у_29', 'співпраці_30', 'з_31', 'україною_32', 'та_33', 'готовності_34', 'здійснити_35', 'відкладений_36', 'візит_37', 'до_38'

## Version II
-  Golden - залежності, які проставив парсер stanz-и
- Порівнюю роботу свого парсера на реченнях, які привела до необхідного вигляду з допомогою stanza (('head', None))
- В цій версії фічі кодую по одній, оскільки в conllu - 'feats', OrderedDict([('Animacy', 'Inan'), ('Case', 'Nom'), ('Gender', 'Masc'), ('Number', 'Sing')]), а в stanza - "Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing"

In [20]:
# use stanza to transform test sentenses
def stanza_sentence(sentence):
    transformed_tree = []
    doc = stanza_nlp(sentence)
    for sent in doc.sentences:
        for word in sent.words:
            transformed_tree.append(OrderedDict([('id', int(word.id)), ('form',  word.text), ('lemma',word.lemma),
                                     ('upostag', word.upos), ('xpostag', None),
                                     ('feats', word.feats), ('head', None), ('deprel', None),
                                     ('deps', None), ('misc', None)]))
    return transformed_tree

In [21]:
# stanza feats format is string ("Case=Nom|Gender=Fem|Number=Sing")
def extract_stanza_features(stack, queue):
    features = dict()

    if len(stack) > 0:
        stack_top = stack[-1]
        features["s0-word"] = stack_top["form"]
        features["s0-lemma"] = stack_top["lemma"]
        features["s0-tag"] = stack_top["upostag"]
         # фічі для ост-го слова в стеці
        if stack_top["feats"] != None:
            fts = stack_top["feats"].split("|")
            for i in fts:
                idx = i.find("=")
                features["s0-"+ i[:idx]] = i[idx+1:]
        else:
            features["s0-feats"] = False
    if len(stack) > 1:
        features["s1-tag"] = stack[-2]["upostag"]
        if stack[-2]["feats"] != None:
            st_fts = stack[-2]["feats"].split("|")
            for i in st_fts:
                idx = i.find("=")
                features["s1-"+ i[:idx]] = i[idx+1:]
        else:
            features["s1-feats"] = False
    if queue:
        queue_top = queue[0]
        features["q0-word"] = queue_top["form"]
        features["q0-lemma"] = queue_top["lemma"]
        features["q0-tag"] = queue_top["upostag"]
        # фічі для 1-го слова в черзі
        if queue_top["feats"] != None:
            qt_fts = queue_top["feats"].split("|")
            for i in qt_fts:
                idx = i.find("=")
                features["q0-"+ i[:idx]] = i[idx+1:]
        else:
            features["q0-feats"] = False
    if len(queue) > 1:
        queue_next = queue[1]
        features["q1-word"] = queue_next["form"]
        features["q1-tag"] = queue_next["upostag"]
        # фічі для 2-го слова в черзі
        if queue_next["feats"] != None:
            q_fts = queue_next["feats"].split("|")
            for i in q_fts:
                idx = i.find("=")
                features["q1-"+ i[:idx]] = i[idx+1:]
        else:
            features["q1-feats"] = False
    if len(queue) > 2:
        features["q2-tag"] = queue[2]["upostag"]
    if len(queue) > 3:
        features["q3-tag"] = queue[3]["upostag"]
    return features

In [22]:
def dep_parse_with_stanza_feats(sent, oracle, vectorizer):
    stack, queue, relations = [ROOT], sent[:], []
    while queue or stack:
        if stack and not queue:
            stack.pop()
        else:
            features = extract_stanza_features(stack, queue)
            action = oracle.predict(vectorizer.transform([features]))[0]
            if action == Actions.SHIFT:
                stack.append(queue.pop(0))
            elif action == Actions.REDUCE:
                stack.pop()
            elif action == Actions.LEFT:
                relations.append((stack[-1]["id"], queue[0]["id"]))
                stack.pop()
            elif action == Actions.RIGHT:
                relations.append((queue[0]["id"], stack[-1]["id"]))
                stack.append(queue.pop(0))
            else:
                print("Unknown action.")
    return sorted(relations)

In [23]:
def extract_features_v2(stack, queue):
    features = dict()

    if len(stack) > 0:
        stack_top = stack[-1]
        features["s0-word"] = stack_top["form"]
        features["s0-lemma"] = stack_top["lemma"]
        features["s0-tag"] = stack_top["upostag"]
         # фічі для ост-го слова в стеці
        if stack_top["feats"] != None:
            for k, v in stack_top["feats"].items():
                features["s0-"+ k] = v
        else:
            features["s0-feats"] = False
    if len(stack) > 1:
        features["s1-tag"] = stack[-2]["upostag"]
        if stack[-2]["feats"] != None:
            for k, v in stack[-2]["feats"].items():
                features["s1-"+ k] = v
        else:
            features["s1-feats"] = False
    if queue:
        queue_top = queue[0]
        features["q0-word"] = queue_top["form"]
        features["q0-lemma"] = queue_top["lemma"]
        features["q0-tag"] = queue_top["upostag"]
        # фічі для 1-го слова в черзі
        if queue_top["feats"] != None:
            for k, v in queue_top["feats"].items():
                features["q0-"+ k] = v
        else:
            features["q0-feats"] = False
    if len(queue) > 1:
        queue_next = queue[1]
        features["q1-word"] = queue_next["form"]
        features["q1-tag"] = queue_next["upostag"]
        # фічі для 2-го слова в черзі
        if queue_next["feats"] != None:
            for k, v in queue_next["feats"].items():
                features["q1-"+ k] = v
        else:
            features["q1-feats"] = False
    if len(queue) > 2:
        features["q2-tag"] = queue[2]["upostag"]
    if len(queue) > 3:
        features["q3-tag"] = queue[3]["upostag"]
    return features

In [25]:
def get_data_v2(tree):
    features, labels = [], []
    stack, queue, relations = [ROOT], tree[:], []
    
    while queue or stack:
        action = oracle(stack if len(stack) > 0 else None,
                       queue[0] if len(queue) > 0 else None,
                       relations)
        features.append(extract_features_v2(stack, queue))
        labels.append(action.value)
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            stack.append(queue.pop(0))
        else:
            print("Unknown action.")
    return features, labels

In [26]:
train_feats, train_lbls = [], []
for tree in train_trees:
    tree_feats, tree_lbls = get_data_v2([t for t in tree if type(t["id"])==int])
    train_feats += tree_feats
    train_lbls += tree_lbls

print(len(train_feats), len(train_lbls))

190298 190298


In [27]:
vec_v2 = vectorizer.fit(train_feats)
print("\nTotal number of features:", len(vec_v2.get_feature_names()))


Total number of features: 111397


In [28]:
train_feats_vectorized = vec_v2.transform(train_feats)
print(len(train_feats_vectorized.toarray()))

190298


In [29]:
lrc_v2 = LogisticRegression(random_state=42, solver="saga", multi_class="multinomial", max_iter=1000, verbose=1)
lrc_v2.fit(train_feats_vectorized, train_lbls)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 404 seconds


/Users/escapistdreamer/anaconda3/envs/spacy_nlp/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.7min finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='saga', tol=0.0001, verbose=1,
                   warm_start=False)

In [30]:
# version 2:
total, tp, full_match = 0, 0, 0
for sentence in corpus:
    transformed_tree = stanza_sentence(sentence)
    stanza_golden = golden_tree(sentence)
    tree = [t for t in transformed_tree if type(t["id"])==int]
    golden = [(node["id"], node["head"]) for node in stanza_golden]
    predicted = dep_parse_with_stanza_feats(tree, lrc_v2, vec_v2)
    total += len(tree)
    print("Sentence:", [i["form"]+"_"+str(i["id"]) for i in transformed_tree])
    print("Golden:", golden)
    print("Predicted:", predicted)
    print("========================")
    tp += len(set(golden).intersection(set(predicted)))
    if set(golden) == set(predicted):
        full_match += 1

print("Total:", total)
print("Correctly defined:", tp)
print("UAS:", round(tp/total, 2))
print("Full match:", round(full_match/len(corpus), 2))

Sentence: ['Цінова_1', 'лихоманка_2', 'у_3', 'царині_4', 'енергоресурсів_5', 'засвідчує_6', ',_7', 'що_8', 'світова_9', 'економіка_10', 'переживає_11', 'не_12', 'найкращі_13', 'часи_14', '._15']
Golden: [(1, 2), (2, 6), (3, 4), (4, 2), (5, 4), (6, 0), (7, 11), (8, 11), (9, 10), (10, 11), (11, 6), (12, 13), (13, 14), (14, 11), (15, 6)]
Predicted: [(1, 2), (2, 0), (2, 6), (3, 4), (4, 2), (4, 6), (5, 4), (6, 0), (7, 11), (8, 11), (9, 10), (10, 11), (11, 6), (12, 14), (13, 14), (14, 11), (15, 6)]
Sentence: ['Проте_1', 'усвідомлення_2', 'цього_3', 'факту_4', 'не_5', 'завадило_6', 'верховному_7', 'представнику_8', 'ЄС_9', 'у_10', 'закордонних_11', 'справах_12', 'та_13', 'політиці_14', 'безпеки_15', 'Жозепу_16', 'Боррелю_17', 'під_18', 'час_19', 'спілкування_20', 'з_21', 'Дмитром_22', 'Кулебою_23', 'наголосити_24', 'на_25', 'зацікавленості_26', 'Європейського_27', 'Союзу_28', 'у_29', 'співпраці_30', 'з_31', 'Україною_32', 'та_33', 'готовності_34', 'здійснити_35', 'відкладений_36', 'візит_37',

### Conclusions
- З 2-х результатів можна зробити такі висновки:
- 1) Додавання граматичних ознак покращує роботу парсера. Це видно з Total: 158, Correctly defined: 126, UAS: 0.8, де кожна граматична ознака була закодована окремо, а сам набір граматичних ознак був одинаковим для тренувальних та тестових даних.
- 2) Хоча ці результати теж відносні, оскільки за golden взято залежності, які проставив парсер stanz-и.